# 给所有药物生成训练数据
使用所有病人的数据，生成所有药物的训练数据。

训练数据按药物生成，不进行负采样，使用所用负样本。训练数据量151*9960 = 1503960。

验证集按病人生成，使用所有药物。验证集数据量151\*num_val_visits = 151\*40 = 6040。

In [1]:
from generate_finetune_data import generate_finetune_data

generate_finetune_data(None, num_val_visits=40, file_name='data4LLM_CONCISE_NOTE.csv')

Loaded data from data4LLM_CONCISE_NOTE.csv


Generating finetune data: 100%|██████████| 151/151 [03:39<00:00,  1.46s/it]


Generate finetune data for all successfully! Train Length: 1430121=219842+1210279, Val Length: 6040=940+5100


('../data/finetune_data/all.json', 940, 5100)

# 给一部分药物生成训练数据
使用所有病人的数据，生成所有药物的训练数据。

训练数据按药物生成，不进行负采样，使用所用负样本。训练数据量151*9960 = 1503960。

验证集按药物生成，使用固定数量的病人。验证集数据量n_drugs\*num_val_examples = 5\*200 = 1000。

In [1]:
from generate_finetune_data import generate_finetune_data

generate_finetune_data(['Acetaminophen', 'Pantoprazole', 'Metoprolol'], num_val_examples=200, file_name='data4LLM_CONCISE_NOTE.csv')

Loaded data from data4LLM_CONCISE_NOTE.csv


Generating finetune data: 100%|██████████| 3/3 [00:04<00:00,  1.47s/it]


Generate finetune data for Acetaminophen-Pantoprazole-Metoprolol successfully! Train Length: 28413=21240+7173, Val Length: 600=390+210


('../data/finetune_data/Acetaminophen-Pantoprazole-Metoprolol.json', 390, 210)

# 将药物分成lora_num个组，每组生成训练数据和验证集。

In [ ]:
import sys
sys.path.append('../')
import utils
from generate_finetune_data import generate_finetune_data

lora_num = 10

base_model = 'llama-2-7b'

_, med_names = utils.load_data(mode='val', file_name='data4LLM_CONCISE_NOTE.csv')

split_point_list = [0]
split_list = []

num_val_examples = 100   # 每个药物的验证集大小，包含正样本和负样本

for i in range(lora_num):
    split_point = int(len(med_names) / lora_num * (i + 1))
    split_point_list.append(split_point)

for i in range(lora_num):
    split_list.append(med_names[split_point_list[i]:split_point_list[i + 1]])

for list_idx in range(lora_num):
    file_name, val_1, val_2 = generate_finetune_data(split_list[list_idx], num_val_examples=num_val_examples, file_name='data4LLM_CONCISE_NOTE.csv')
    para_str = f'--learning_rate 5e-4 --base_model {base_model} --num_epochs 50 --med_num {num_val_examples} --val_set_size {val_1 + val_2} --batch_size 64 --micro_batch_size 2 --train_on_inputs 0  --resume_from_checkpoint 0 --data_path {file_name} --output_dir ../output/lora_{split_point_list[list_idx]}_{split_point_list[list_idx + 1]}_2/'

    start_idx = split_point_list[list_idx]
    end_idx = split_point_list[list_idx + 1]

# 根据lora_num的同时，限制负样本比例生成对应数据集

In [ ]:
import sys
sys.path.append('../')
import utils
from generate_finetune_data import generate_finetune_data

lora_num = 1

base_model = 'llama-2-13b-chat'

negative_ratio = 6

_, med_names = utils.load_data(mode='val', file_name='data4LLM_CONCISE_NOTE.csv')

split_point_list = [0]
split_list = []

for i in range(lora_num):
    split_point = int(len(med_names) / lora_num * (i + 1))
    split_point_list.append(split_point)

for i in range(lora_num):
    split_list.append(med_names[split_point_list[i]:split_point_list[i + 1]])

for list_idx in range(lora_num):
    file_name, val_1, val_2 = generate_finetune_data(split_list[list_idx], negative_ratio=negative_ratio ,num_val_visits=40, file_name='data4LLM_CONCISE_NOTE.csv')


# 对生成的数据集进行查看

In [ ]:
import json

file_name = '../data/finetune_data/all_ratio_10.json'

with open(file_name, 'r') as f:
    json_data = json.load(f)

for idx in range(len(json_data)):
    dict_data = json_data[idx]
    input_str = dict_data['input']
    last_colon_index = input_str.rfind(': ')
    last_period_index = input_str.rfind('.')

    med_name = input_str[last_colon_index + 2:last_period_index]
    output = dict_data['output']

# 给每个药物生成一个训练数据

In [ ]:
from generate_finetune_data import generate_finetune_data

generate_finetune_data('Valsartan',None, 2)

# 打印原始数据的信息

In [ ]:
import sys
sys.path.append('../')
import utils

data_train, med_names, med_weights = utils.load_data(mode='train')
data_train